## Programming part of Homework 5 (Data Structures, Fall 2022)

## Name: 歐佳昀

### Programming problem 
This exercise is about to implement a ***skip list** using *linked list structure* with **Python**. Sample definitions for the list node and skip list classes are given as below. Recall the discussion in class. Each node has an entry with two attributes, *key* and *name*, where key is an integer and name is a string, as well as four links: *prev*, *next*, *up*, and *down*. In order to manage multiple lists used in the skip list, we use the built-in data structure, ***list***, of Python. The methods in the implemented skip list include:
1. `addEmptyList()`: This is to pad the skip list when the number of copies of the inserted node is more than the height (number of lists used) of the current skip list when inserting nodes;
2. `search(v)`: This method searches the skip list with the given node $v$ using the key;
3. `delete(v)`: This method will delete the given node $v$ from the skip list;
4. `insert(v)`: This is to insert the given node $v$ to the skip list.
The Python program starts with function `create\_SkipLists()` which reads the input file, `inFile.txt`. In the input file, each line contains only one entry: key and string and as follows:

```
10 mary
25 john
35 mars
50 lowe
```

An example of input file will also be provided.

In [1]:
import re
import random

# Two special values for boundary nodes
PLUS_INF = 99999
MINUS_INF = -99999

# Node class definition: a quadraic node having four links  
class SLnode:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.up = None
        self.down = None
        self.next = None
        self.prev = None

    def getKey(self):
        return self.key

    def getItem(self):
        return self.item

    def getNext(self):
        return self.next

    def getPrev(self):
        return self.prev

    def getUp(self):
        return self.up

    def getDown(self):
        return self.down

    def hasNext(self):
        return (self.next!=None)

    def hasPrev(self):
        return (self.prev!=None)

    def setKey(self, key):
        self.key = key

    def setItem(self, item):
        self.item = item

    def setNext(self, p):
        self.next = p

    def setPrev(self, p):
        self.prev =p
 
    def setUp(self, p):
        self.up = p

    def setDown(self, p):
        self.down = p

# List class definition used in the skip list 
class SLlist:
    def __init__(self):
        self.leftDummy=SLnode(MINUS_INF,"")
        self.rightDummy=SLnode(PLUS_INF,"")
        self.leftDummy.setNext(self.rightDummy)
        self.rightDummy.setPrev(self.leftDummy)
        self.size = 0
        self.insert_cursor = self.getleftDummy()

    def getleftDummy(self):
        return self.leftDummy

    def getrightDummy(self):
        return self.rightDummy

    def getSize(self):
        return self.size

    def increaseSize(self):
        self.size=self.size + 1

    def decreaseSize(self):
        self.size=self.size - 1

    '''
    method insertAfter(self, p, SLnode): insert a node in the list after node p
    '''
    def insertAfter(self, p, SLnode):
        # p-> sLnode -> tmp
        tmp = p.next
        p.next = SLnode
        SLnode.prev = p
        SLnode.next = tmp
        tmp.prev = SLnode
        

    '''
    method print_List(self): print the content of the list
    '''
    def print_List(self):
        cur = self.leftDummy
        while (cur.key != PLUS_INF):
            print("({},{})".format(cur.key, cur.item), end='')
            cur = cur.next
        print("({},)".format(PLUS_INF))


# Skip list definition: a list of lists is used
class Skip_Lists:
    def __init__(self):
        self.S=[SLlist()]

    def getLists(self):
        return self.S

    # use the number of nodes in the bottom list to denote the Size
    def getSize(self):
        return self.S[0].getSize()

    # use Height to denote the number of lists used in the skip list
    def getHeight(self):
        return len(self.S)

    def isEmpty(self):
        return ((self.getHeight()==1) and (self.getSize()==0))

    # Derive the top list in the skip list
    def getTopList(self):
        return self.S[self.getHeight()-1]

    '''
    method getTopleft(self): get the topleft node in the skip list
    '''
    # Derive the topleft node in the skip list
    def getTopleft(self):
        return self.S[len(self.S)-1].leftDummy
        
    '''
    method addEmptyList(): padding the skip list when the number of copies of the inserted node
    is more than the height of the current  skip list
    '''
    def addEmptyList(self):
        self.S.append(SLlist())
        self.S[len(self.S)-1].leftDummy.down = self.S[len(self.S)-2].leftDummy

        
    '''
    method search(node): search the skip list with the given node using the key
    '''
    def search(self, node):
        
        cur = self.getTopleft()
        
        while cur is not None:
            if cur.next.key > node.key:
                cur = cur.down
            elif cur.next.key == node.key:
                print("Serch (key = {}) in Skip-List item is : ({},{})".format(node.key,node.key,node.item))
                return cur.next.item
            else:  # cur.next is not None and cur.next.val < target
                cur = cur.next

        return None

    '''
    method delete(node): delete the given node from the skip list
    '''
    def delete(self, node):
        
        cur = self.getTopleft()
        lvl = len(self.S)-1
        
        while cur is not None:
            if cur.next is None or cur.next.key > node.key:
                cur = cur.down
                lvl -= 1
            elif cur.next.key == node.key:
                de = cur.next
                flag = False
                while de is not None:
                    flag = True
                    de.prev.next = de.next
                    de.next.prev = de.prev
                    de = de.down
                    self.S[lvl].decreaseSize()
                    lvl -= 1
                while (True):
                    if (self.S[len(self.S)-1].leftDummy.next != self.S[len(self.S)-1].rightDummy): #top isn't empty list
                        self.addEmptyList()
                        return
                    elif (self.isEmpty() is False): # top is empty list but not the first list ，remove unnecessary list 
                        self.S.pop()
                    elif flag: # delete element is the last key
                        return
                    else:
                        break
            else:
                cur = cur.next
        # print("Key not found in the skip lists and will not perform the deletion")

        
    '''
    method insert(node): insert the given node to the skip list
    '''
    def insert(self, node):
        
        if (self.isEmpty() == False): #remove the first one
            self.S.pop()
            
        cur = self.getTopleft()
        stack = []
        
        while True:
            if cur.next.key < node.key:
                cur = cur.next
            elif cur.down != None:
                stack.append(cur)
                cur = cur.down
            else:
                if (cur.key == node.key or cur.next.key == node.key):
                    self.addEmptyList()
                    # print("Key found in the skip lists and will not inert the new node")
                    return
                self.S[0].increaseSize()
                # cur <-> node <-> cur.next
                self.S[0].insertAfter(cur, node)
                n = node
                flag = True
                lvl = 1

                while coin_tossing() and flag:

                    lvl += 1
                    if (len(stack) == 0):

                        self.addEmptyList()
                        now = self.S[len(self.S)-1].leftDummy
                        flag = False

                    else:
                        now = stack.pop()

                    self.S[lvl-1].increaseSize()
                    _node = SLnode(node.key, node.item)
                    # now -> _node -> now.next
                    self.S[lvl-1].insertAfter(now, _node)
                    _node.down = n
                    n = _node

                break
        self.addEmptyList()


'''
function for coin tossing with the number of heads returned
'''
def coin_tossing():
    p = random.choice([0, 1])
    if(p == 1):
        return True
    else:
        return False
    
'''
function for reading lines (entries) in the input text file into a list of strings
'''
def read_lines():
    with open('inFile.txt', "r+") as f:
        entryListA = [x.strip()for x in f.readlines()]
    f.close()

    return entryListA
    
'''
function for starting the task
'''
def create_SkipLists():
    #
    # read the input information from the default input text file into an
    # entry list, entry_list
    #
    entry_list=read_lines()
    #
    # initiating a skip list object SL
    #
    SL=Skip_Lists()
    for index in range(0, len(entry_list)):
        # splitting the string by " " symbol for deriving the entry                                       
        pairs = re.split(" ",entry_list[index])
        # making a new node for the entry
        newnode=SLnode(int(pairs[0]), pairs[1])
        # inserting the new node to the skip list SL
        SL.insert(newnode)

    #--------------dynamic operations with result printed -----------------------------------
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("Insert (88, luke)")
    SL.insert(SLnode(88, "luke"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (40, kite)")
    SL.delete(SLnode(40, "kite"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("Insert (27, eric)")
    SL.insert(SLnode(27, "eric"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (45, lisa)")
    SL.delete(SLnode(45, "lisa"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (27, luis)")
    SL.delete(SLnode(27, "luis"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (8, kids)")
    SL.delete(SLnode(8, "kids"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (88, luke)")
    SL.delete(SLnode(88, "luke"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    return

print(read_lines())
print('-----------------------------------------------')
create_SkipLists()

['27 luis', '8 kids', '40 kite']
-----------------------------------------------
(-99999,)(8,kids)(27,luis)(40,kite)(99999,)
(-99999,)(8,kids)(27,luis)(40,kite)(99999,)
(-99999,)(8,kids)(40,kite)(99999,)
(-99999,)(99999,)
Insert (88, luke)
(-99999,)(8,kids)(27,luis)(40,kite)(88,luke)(99999,)
(-99999,)(8,kids)(27,luis)(40,kite)(88,luke)(99999,)
(-99999,)(8,kids)(40,kite)(99999,)
(-99999,)(99999,)
delete (40, kite)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(99999,)
(-99999,)(99999,)
Insert (27, eric)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(99999,)
(-99999,)(99999,)
delete (45, lisa)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(27,luis)(88,luke)(99999,)
(-99999,)(8,kids)(99999,)
(-99999,)(99999,)
delete (27, luis)
(-99999,)(8,kids)(88,luke)(99999,)
(-99999,)(8,kids)(88,luke)(99999,)
(-99999,)(8,kids)(99999,)
(-99999,)(99999,)
delete (8, kids)
(-